In [ ]:
REPO_FOLDER = "../src"
RESULTS_FOLDER = "/path/to/results/folder"

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from typing import Sequence, Optional
import os
import json

import numpy as np

sys.path.append(REPO_FOLDER)

from blockflow.simulator import compute_other_scores, FakeContractParameters, save
from blockflow.model import ModelFactory

In [ ]:
def _rescore(folder_path: str, dp_round: int, reward_pool: np.ndarray, contract_parameters: FakeContractParameters, num_dishonest_clients: Optional[int] = None, zero_others: Optional[bool] = None, out_folder_path: Optional[str] = None):
    if out_folder_path is None:
        out_folder_path = folder_path
    with np.load(os.path.join(folder_path, f"data_dp_round_{dp_round}.npz"), "r", allow_pickle=True) as data:
        score_matrix = data['score_matrix']  
        ground_truth_scores = data['ground_truth_scores']
    # let the dishonest clients fuck themselves
    # the first num_dishonest_clients are dishonest
    if num_dishonest_clients is not None:
        if zero_others is None:
            raise TypeError("if there are dishonest clients, then zero_others must be specified")
        score_matrix[:num_dishonest_clients, :num_dishonest_clients] = 1.0  # give each-other 1s
        if zero_others:
            score_matrix[:num_dishonest_clients, num_dishonest_clients:] = 0.0  # and give everyone else 0s
    median_model_scores, auditor_scores, overall_scores, refund_amount = compute_other_scores(score_matrix, dp_round, reward_pool, contract_parameters)
    os.makedirs(out_folder_path, exist_ok=True)
    print(f"about to save to {out_folder_path}")
    save(out_folder_path, dp_round, score_matrix, ground_truth_scores, median_model_scores, auditor_scores, overall_scores, refund_amount)

In [ ]:
def rescore(folder_path: str, num_dishonest_clients: Optional[int] = None, zero_others: Optional[bool] = None, out_folder_path: Optional[str] = None):
    with open(os.path.join(folder_path, "contract_parameters.json"), "r") as f:
        contract_parameters = FakeContractParameters(**json.load(f))
    with np.load(os.path.join(folder_path, "data_dp_round_1.npz"), "r", allow_pickle=True) as data:
        num_clients = len(data['score_matrix'])
    reward_pool = np.array(num_clients, dtype=np.float)
    for dp_round in range(1, 1+contract_parameters.num_dp_rounds):
        _rescore(folder_path, dp_round, reward_pool, contract_parameters, num_dishonest_clients, zero_others, out_folder_path)

In [ ]:
def exp_3(exp_name: str):
    base_dir = os.path.join(RESULTS_FOLDER, f"{exp_name}/split_sym_eq_50_validation_fraction_0.2/inv_lambda-1.0_epsilon-0.01_apply_dp-True")
    num_dishonest_clients = (1, 12, 24)
    for num_dishonest in num_dishonest_clients:
        for zero_others in (True, False):
            new_out_dir = os.path.join(base_dir, f"scoring_dishonest-{num_dishonest}_zero_others-{zero_others}")
            rescore(base_dir, num_dishonest, zero_others, new_out_dir)

In [ ]:
exp_3("kdd99")

In [ ]:
exp_3("adult")